In [2]:
from tqdm import tqdm_notebook, tnrange
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from skimage.feature import canny
from skimage.filters import sobel,threshold_otsu, threshold_niblack,threshold_sauvola
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries
from scipy import signal

In [3]:
import os
import sys
import random
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import cv2
from PIL import Image
import pdb
from tqdm import tqdm
import seaborn as sns
import os 
from glob import glob

import warnings
warnings.filterwarnings("ignore")

In [10]:
INPUT_PATH = 'E:/DC/钢筋数量AI识别'
DATA_PATH = INPUT_PATH
TRAIN_DATA = os.path.join(DATA_PATH, "train")
TRAIN_MASKS_DATA = os.path.join(DATA_PATH, "train/masks")
TEST_DATA = os.path.join(DATA_PATH, "test")
df = pd.read_csv(DATA_PATH+'/train_labels.csv',engine='python')
path_train = 'E:/DC/钢筋数量AI识别/train/'
path_test = 'E:/DC/钢筋数量AI识别/test/'
train_ids = df.ID.values
df = df.set_index('ID')

In [11]:
df.head()

,Detection,AB
ID,,
DBA90206.jpg,311 707 472 842,NaN
DBA90206.jpg,351 631 452 721,NaN
DBA90206.jpg,438 473 519 563,NaN
DBA90206.jpg,420 553 557 688,NaN
DBA90206.jpg,501 446 641 607,NaN


In [12]:
def get_filename(image_id, image_type):
    check_dir = False
    if "Train" == image_type:
        data_path = TRAIN_DATA
    elif "mask" in image_type:
        data_path = TRAIN_MASKS_DATA
    elif "Test" in image_type:
        data_path = TEST_DATA
    else:
        raise Exception("Image type '%s' is not recognized" % image_type)

    if check_dir and not os.path.exists(data_path):
        os.makedirs(data_path)

    return os.path.join(data_path, "{}".format(image_id))

def get_image_data(image_id, image_type, **kwargs):
    img = _get_image_data_opencv(image_id, image_type, **kwargs)
    img = img.astype('uint8')
    return img

def _get_image_data_opencv(image_id, image_type, **kwargs):
    fname = get_filename(image_id, image_type)
    img = cv2.imread(fname)
    assert img is not None, "Failed to read image : %s, %s" % (image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [13]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

# https://github.com/ternaus/TernausNet/blob/master/Example.ipynb

In [14]:
def mask_overlay(image, mask):
    """
    Helper function to visualize mask
    """
    mask = mask.astype(np.uint8)
    weighted_sum = cv2.addWeighted(mask, 0.75, image, 0.5, 0.)
    img = image.copy()
    ind = mask[:, :, 1] > 0    
    img[ind] = weighted_sum[ind]    
    return img

In [15]:
nImg = 32  #no. of images that you want to display
np.random.seed(42)
if df.index.name == 'ImageId':
    df = df.reset_index()
if df.index.name != 'ImageId':
    df = df.set_index('ImageId')
    
_train_ids = list(train_ids)
np.random.shuffle(_train_ids)
# _train_ids = _train_ids[:nImg]
tile_size = (256, 256)
n = 8
alpha = 0.3
# m = int(np.ceil(len(_train_ids) * 1.0 / n))
m = int(np.ceil(nImg * 1.0 / n))
complete_image = np.zeros((m*(tile_size[0]+2), n*(tile_size[1]+2), 3), dtype=np.uint8)
complete_image_masked = np.zeros((m*(tile_size[0]+2), n*(tile_size[1]+2), 3), dtype=np.uint8)

counter = 0
for i in range(m):
    ys = i*(tile_size[1] + 2)
    ye = ys + tile_size[1]
    j = 0
    while j < n:
        counter += 1
        all_masks = np.zeros((768, 768))
        xs = j*(tile_size[0] + 2)
        xe = xs + tile_size[0]
        image_id = _train_ids[counter]
        if str(df.loc[image_id,'EncodedPixels'])==str(np.nan):
            continue
        else:
            j += 1
        img = get_image_data(image_id, 'Train')
        
        try:
            img_masks = df.loc[image_id,'EncodedPixels'].tolist()
            for mask in img_masks:
                all_masks += rle_decode(mask)
            all_masks = np.expand_dims(all_masks,axis=2)
            all_masks = np.repeat(all_masks,3,axis=2).astype('uint8')*255
            
            img_masked = mask_overlay(img, all_masks)
            
#             img_masked =  cv2.addWeighted(img, alpha, all_masks, 1 - alpha,0)

            img = cv2.resize(img, dsize=tile_size)
            img_masked = cv2.resize(img_masked, dsize=tile_size)

            img = cv2.putText(img, image_id, (5,img.shape[0] - 5), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), thickness=2)
            complete_image[ys:ye, xs:xe, :] = img[:,:,:]

            img_masked = cv2.putText(img_masked, image_id, (5,img.shape[0] - 5), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), thickness=2)
            complete_image_masked[ys:ye, xs:xe, :] = img_masked[:,:,:]

        except Exception as e:
            all_masks = rle_decode(df.loc[image_id,'EncodedPixels'])
            all_masks = np.expand_dims(all_masks,axis=2)*255
            all_masks = np.repeat(all_masks,3,axis=2).astype('uint8')
            
            img_masked = mask_overlay(img, all_masks)        
#             img_masked =  cv2.addWeighted(img, alpha, all_masks, 1 - alpha,0)
    #         img_masked = cv2.bitwise_and(img, img, mask=all_masks)

            img = cv2.resize(img, dsize=tile_size)
            img_masked = cv2.resize(img_masked, dsize=tile_size)

            img = cv2.putText(img, image_id, (5,img.shape[0] - 5), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), thickness=2)
            complete_image[ys:ye, xs:xe, :] = img[:,:,:]
#             pdb.set_trace()

            img_masked = cv2.putText(img_masked, image_id, (5,img.shape[0] - 5), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), thickness=2)
            complete_image_masked[ys:ye, xs:xe, :] = img_masked[:,:,:]

        
m = complete_image.shape[0] / (tile_size[0] + 2)
k = 8
n = int(np.ceil(m / k))
for i in range(n):
    plt.figure(figsize=(20, 20))
    ys = i*(tile_size[0] + 2)*k
    ye = min((i+1)*(tile_size[0] + 2)*k, complete_image.shape[0])
    plt.imshow(complete_image[ys:ye,:,:],cmap='seismic')
    plt.title("Training dataset")
    
m = complete_image.shape[0] / (tile_size[0] + 2)
k = 8
n = int(np.ceil(m / k))
for i in range(n):
    plt.figure(figsize=(20, 20))
    ys = i*(tile_size[0] + 2)*k
    ye = min((i+1)*(tile_size[0] + 2)*k, complete_image.shape[0])
    plt.imshow(complete_image_masked[ys:ye,:,:])
    plt.title("Training dataset: Lighter Color depicts ship")

KeyError: 'ImageId'